# Query 2: Top Performing Police Divisions

## Description:

The goal of Query 2 is to identify the top three police precincts in Los Angeles with the highest case resolution rates for each year. The analysis includes:

- Calculating the closed case rate for each precinct and year.

- Ranking the precincts annually based on their resolution rates.

- Selecting the top three precincts per year.

This task is implemented using both the DataFrame API and the SQL API in PySpark. The results are saved as CSV files for further analysis. The execution time for both implementations is measured and compared to evaluate their performance.



In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, rank, extract, to_timestamp, to_date, year
from pyspark.sql.window import Window

def dataframe_implementation(spark, form):
    crime_data_2010_2019 = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True, inferSchema=True)
    crime_data_2020_present = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv", header=True, inferSchema=True)
    mid_time = time.time()
    combined_data = (
        crime_data_2010_2019.select(
            year(to_timestamp(col("DATE OCC"), "MM/dd/yyyy hh:mm:ss a")).alias("year"),
            col("AREA NAME").alias("precinct"),
            when((col("Status Desc") != "UNK") & (col("Status Desc") != "Invest Cont"), 1).otherwise(0).alias("closed_case")
        ).union(
            crime_data_2020_present.select(
                year(to_timestamp(col("DATE OCC"), "MM/dd/yyyy hh:mm:ss a")).alias("year"),
                col("AREA NAME").alias("precinct"),
                when((col("Status Desc") != "UNK") & (col("Status Desc") != "Invest Cont"), 1).otherwise(0).alias("closed_case")
            )
        )
    )
    processed_data = (
        combined_data
        .groupBy("year", "precinct")
        .agg(
            (count(when(col("closed_case") == 1, 1)) * 100.0 / count("*")).alias("closed_case_rate"),
            count("*").alias("total_cases")
        )
    )
    window_spec = Window.partitionBy("year").orderBy(col("closed_case_rate").desc())
    ranked_data = processed_data.withColumn("#", rank().over(window_spec))
    top_3_precincts_per_year = (
        ranked_data
        .filter(col("#") <= 3)
        .select("year", "precinct", "closed_case_rate", "#")
        .orderBy("year", "#")
    )
    top_3_precincts_per_year.show(n=50, truncate=False)
    return time.time(), mid_time, top_3_precincts_per_year

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2917,application_1732639283265_2876,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
def sql_implementation(spark, form = 'csv'):
    if form == 'csv':
        crime_data_2010_2019 = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True, inferSchema=True)
        crime_data_2020_present = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv", header=True, inferSchema=True)
    else:
        crime_data_2010_2019 = spark.read.parquet("s3://groups-bucket-dblab-905418150721/group21/Crime_Data_from_2010_to_2019_20241101.parquet", header=True, inferSchema=True)
        crime_data_2020_present = spark.read.parquet("s3://groups-bucket-dblab-905418150721/group21/Crime_Data_from_2020_to_Present_20241101.parquet", header=True, inferSchema=True)
    crime_data_2010_2019.createOrReplaceTempView("crime_data_2010_2019")
    crime_data_2020_present.createOrReplaceTempView("crime_data_2020_present")
    mid_time = time.time()
# SQL Query for top 3 precincts
    sql_query = """
    WITH combined_data AS (
        SELECT 
            to_date(`DATE OCC`, 'MM/dd/yyyy hh:mm:ss a') AS date,
            `AREA NAME` AS precinct,
            `Status Desc`
        FROM crime_data_2010_2019
        UNION ALL
        SELECT 
            to_date(`DATE OCC`, 'MM/dd/yyyy hh:mm:ss a') AS date,
            `AREA NAME` AS precinct,
            `Status Desc`
        FROM crime_data_2020_present
    ),
    processed_data AS (
        SELECT 
            YEAR(date) AS year,
            precinct,
            COUNT(CASE WHEN (`Status Desc` <> 'UNK' AND `Status Desc` <> 'Invest Cont') THEN 1 END) * 100.0 / COUNT(*) AS closed_case_rate,
            RANK() OVER (PARTITION BY YEAR(date) ORDER BY COUNT(CASE WHEN (`Status Desc` <> 'UNK' AND `Status Desc` <> 'Invest Cont') THEN 1 END) * 100.0 / COUNT(*) DESC) AS rank
        FROM combined_data
        GROUP BY YEAR(date), precinct
    )
    SELECT 
        year, 
        precinct, 
        closed_case_rate, 
        rank
    FROM processed_data
    WHERE rank <= 3
    ORDER BY year, rank
    """

    top_3_precincts_per_year_sql = spark.sql(sql_query)

    top_3_precincts_per_year_sql.show(n=50, truncate=False)
    return time.time(), mid_time, top_3_precincts_per_year_sql

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
import time

def measure_execution_time(func, question, form = 'csv'):
    spark = SparkSession.builder \
                    .appName(question) \
                    .getOrCreate()
    spark.catalog.clearCache()
    start_time = time.time()
    end_time, mid_time, df  = func(spark, form)
    return end_time - mid_time, mid_time - start_time, df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df_api_time, df_read_api_time, _ = measure_execution_time(dataframe_implementation, "Query 2a Dataframe")

print(f"DataFrame API Execution Time: {df_api_time} seconds")
print(f"DataFrame API Read Time: {df_read_api_time} seconds")
print(f"DataFrame API Total Time: {df_api_time + df_read_api_time} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+------------------+---+
|year|precinct   |closed_case_rate  |#  |
+----+-----------+------------------+---+
|2010|Rampart    |32.84713448949121 |1  |
|2010|Olympic    |31.515289821999087|2  |
|2010|Harbor     |29.36028339237341 |3  |
|2011|Olympic    |35.0400600901352  |1  |
|2011|Rampart    |32.496447181430604|2  |
|2011|Harbor     |28.513362463164313|3  |
|2012|Olympic    |34.29708533302119 |1  |
|2012|Rampart    |32.46000463714352 |2  |
|2012|Harbor     |29.50958584895668 |3  |
|2013|Olympic    |33.58217940999398 |1  |
|2013|Rampart    |32.1060382916053  |2  |
|2013|Harbor     |29.723638951488553|3  |
|2014|Van Nuys   |32.0215235281705  |1  |
|2014|West Valley|31.49754809505847 |2  |
|2014|Mission    |31.22493985565357 |3  |
|2015|Van Nuys   |32.265140677157845|1  |
|2015|Mission    |30.463762673676303|2  |
|2015|Foothill   |30.353001803658852|3  |
|2016|Van Nuys   |32.194518462124094|1  |
|2016|West Valley|31.40146437042384 |2  |
|2016|Foothill   |29.9086472281316

In [8]:
sql_api_time, sql_read_api_time, _ = measure_execution_time(sql_implementation, "Query 2a SQL")

print(f"SQL API Execution Time: {sql_api_time} seconds")
print(f"SQL API Read Time: {sql_read_api_time} seconds")
print(f"SQL API Total Time: {sql_api_time + sql_read_api_time} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+-----------------+----+
|year|precinct   |closed_case_rate |rank|
+----+-----------+-----------------+----+
|2010|Rampart    |32.84713448949121|1   |
|2010|Olympic    |31.51528982199909|2   |
|2010|Harbor     |29.36028339237341|3   |
|2011|Olympic    |35.04006009013520|1   |
|2011|Rampart    |32.49644718143060|2   |
|2011|Harbor     |28.51336246316431|3   |
|2012|Olympic    |34.29708533302119|1   |
|2012|Rampart    |32.46000463714352|2   |
|2012|Harbor     |29.50958584895668|3   |
|2013|Olympic    |33.58217940999398|1   |
|2013|Rampart    |32.10603829160530|2   |
|2013|Harbor     |29.72363895148855|3   |
|2014|Van Nuys   |32.02152352817050|1   |
|2014|West Valley|31.49754809505847|2   |
|2014|Mission    |31.22493985565357|3   |
|2015|Van Nuys   |32.26514067715784|1   |
|2015|Mission    |30.46376267367630|2   |
|2015|Foothill   |30.35300180365885|3   |
|2016|Van Nuys   |32.19451846212410|1   |
|2016|West Valley|31.40146437042384|2   |
|2016|Foothill   |29.9086472281316

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Query 2b Parquet") \
    .getOrCreate()

crime_data_2010_2019 = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True, inferSchema=True)
crime_data_2020_present = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv", header=True, inferSchema=True)

crime_data_2010_2019.write.mode('overwrite').parquet("s3://groups-bucket-dblab-905418150721/group21/Crime_Data_from_2010_to_2019_20241101.parquet")
crime_data_2020_present.write.mode('overwrite').parquet("s3://groups-bucket-dblab-905418150721/group21/Crime_Data_from_2020_to_Present_20241101.parquet")


In [5]:
sql_api_time, sql_read_api_time, final_df = measure_execution_time(sql_implementation, "Query 2b SQL", "parquet")

print(f"SQL API Execution Time: {sql_api_time} seconds")
print(f"SQL API Read Time: {sql_read_api_time} seconds")
print(f"SQL API Total Time: {sql_api_time + sql_read_api_time} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+-----------------+----+
|year|precinct   |closed_case_rate |rank|
+----+-----------+-----------------+----+
|2010|Rampart    |32.84713448949121|1   |
|2010|Olympic    |31.51528982199909|2   |
|2010|Harbor     |29.36028339237341|3   |
|2011|Olympic    |35.04006009013520|1   |
|2011|Rampart    |32.49644718143060|2   |
|2011|Harbor     |28.51336246316431|3   |
|2012|Olympic    |34.29708533302119|1   |
|2012|Rampart    |32.46000463714352|2   |
|2012|Harbor     |29.50958584895668|3   |
|2013|Olympic    |33.58217940999398|1   |
|2013|Rampart    |32.10603829160530|2   |
|2013|Harbor     |29.72363895148855|3   |
|2014|Van Nuys   |32.02152352817050|1   |
|2014|West Valley|31.49754809505847|2   |
|2014|Mission    |31.22493985565357|3   |
|2015|Van Nuys   |32.26514067715784|1   |
|2015|Mission    |30.46376267367630|2   |
|2015|Foothill   |30.35300180365885|3   |
|2016|Van Nuys   |32.19451846212410|1   |
|2016|West Valley|31.40146437042384|2   |
|2016|Foothill   |29.9086472281316

In [7]:
# Define the S3 output path
s3_path = "s3://groups-bucket-dblab-905418150721/group21/top_performing_police_divisions/"

# Save the final results as a CSV
final_df.coalesce(1).write.mode("overwrite").csv(s3_path, header=True)
print(f"Final results saved to: {s3_path}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Final results saved to: s3://groups-bucket-dblab-905418150721/group21/top_performing_police_divisions/